In [2]:
!python -m spacy download en # one time run

     |████████████████████████████████| 11.1MB 4.6MB/s eta 0:00:01    |████▎                           | 1.5MB 1.6MB/s eta 0:00:06
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-cp35-none-any.whl size=11075235 sha256=1049c6441b5eaf041c96d3a5b98a43f2e26b618c07aa68252eb956376ed36ebe
  Stored in directory: /tmp/pip-ephem-wheel-cache-kohkdq51/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/en_core_web_sm -->
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [4]:
df = pd.read_json('abstract.json', lines=True)
df.head()


,abstract
0,The rapid increasing of online information is ...
1,Keywords are very important for any academic p...
2,Paper presents a survey of methods and approac...
3,Every year thousands of academic studies are p...
4,"We describe latent Dirichlet allocation (LDA),..."


In [5]:
df.iloc[0:5]

,abstract
0,The rapid increasing of online information is ...
1,Keywords are very important for any academic p...
2,Paper presents a survey of methods and approac...
3,Every year thousands of academic studies are p...
4,"We describe latent Dirichlet allocation (LDA),..."


In [6]:
data = df.iloc[0:5].values.tolist()
print(data)

[['The rapid increasing of online information is hard to handle.Summaries such as abstracts help us to reduce this problem. Keywords, whichcan be regarded as very short summaries, may help even more. Filteringdocuments by using keywords may save precious time while searching.However, most of the documents do not include keywords. In this paper wepresent a model that extracts keywords from abstracts and titles. This model hasbeen implemented in a prototype system. We have tested our model on a set ofabstracts of Academic papers containing keywords composed by their authors.Results show that keywords extracted from abstracts and titles may be a primarytool for researchers.'], ['Keywords are very important for any academic paper. We propose the Perceptron Training Rule for keyword extraction from titles and abstracts. We present a system for generating keywords which relies on weights of words in a sentence. The system generates keywords from academic research articles by selecting the mo

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words)

[['the', 'rapid', 'increasing', 'of', 'online', 'information', 'is', 'hard', 'to', 'handle', 'summaries', 'such', 'as', 'abstracts', 'help', 'us', 'to', 'reduce', 'this', 'problem', 'keywords', 'whichcan', 'be', 'regarded', 'as', 'very', 'short', 'summaries', 'may', 'help', 'even', 'more', 'by', 'using', 'keywords', 'may', 'save', 'precious', 'time', 'while', 'searching', 'however', 'most', 'of', 'the', 'documents', 'do', 'not', 'include', 'keywords', 'in', 'this', 'paper', 'wepresent', 'model', 'that', 'extracts', 'keywords', 'from', 'abstracts', 'and', 'titles', 'this', 'model', 'hasbeen', 'implemented', 'in', 'prototype', 'system', 'we', 'have', 'tested', 'our', 'model', 'on', 'set', 'ofabstracts', 'of', 'academic', 'papers', 'containing', 'keywords', 'composed', 'by', 'their', 'authors', 'results', 'show', 'that', 'keywords', 'extracted', 'from', 'abstracts', 'and', 'titles', 'may', 'be', 'primarytool', 'for', 'researchers'], ['keywords', 'are', 'very', 'important', 'for', 'any', '

In [8]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

['rapid increasing online information be hard handle summary such abstract help reduce problem keyword whichcan be regard very short summary may help even more use keyword may save precious time search however most document do not include keyword paper wepresent model extract keyword abstract title model hasbeen implement prototype system have test model set ofabstract academic paper contain keyword compose author result show keyword extract abstract title may be primarytool researcher', 'keyword be very important academic paper propose perceptron training rule keyword extraction title abstract present system generate keyword rely weight word sentence system generate keyword academic research article select most relevant keyword compare keyword generate system generate cluster analysis keyword give author analyze result base full keyword match partial keyword match keyword match', 'paper present survey method approach keyword extraction task addition systematization method paper gather

In [ ]:
!pip install -U rake_nltk

In [ ]:
from rake_nltk import Rake
r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
r.extract_keywords_from_text(df['abstract'][3])

In [ ]:
list1=[]
list1=r.get_word_degrees()
print(list1)

In [9]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=1,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)


In [10]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  24.854368932038835 %


In [15]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_topics=6,               # Number of topics
                                      max_iter=50,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=1,            # n docs in each learning iter
                                      evaluate_every = 1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=1, doc_topic_prior=None,
             evaluate_every=1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=6, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [16]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2133.1342778809867
Perplexity:  218.4818013217678
{'batch_size': 1,
 'doc_topic_prior': None,
 'evaluate_every': 1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 50,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'n_topics': 6,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [67]:
# Define Search Param
search_params = {'n_components': [10, 15, 20], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be chang

GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_components': [10, 15, 20], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [68]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))


Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -1562.089940374514
Model Perplexity:  293.2488409229588


In [69]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

In [70]:

# column names
topicnames = ["Topic" + str(i) for i in range(10)]
#print(topicnames)

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
#print(docnames)

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(25).style.applymap(color_green).applymap(make_bold)
df_document_topics
#df_document_topic.Topic0

In [71]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,3,2
1,5,1
2,2,1
3,1,1


In [72]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics          x           y
topic                                                   
2      41.034990        1       1  27.143642   64.281883
3      26.901463        1       2  23.456106   87.020096
5      17.804630        1       3   7.785783   70.357086
1      13.512784        1       4  21.598194  109.817017
7       0.124355        1       5   2.220890  115.873291
6       0.124355        1       6   5.914620   93.157555
9       0.124355        1       7 -11.146079   77.676384
0       0.124355        1       8 -13.747846  100.025269
4       0.124355        1       9  43.122646   80.131462
8       0.124355        1      10  40.511948  102.481453, topic_info=     Category       Freq             Term      Total  loglift  logprob
term                                                                  
92    Default  25.000000          keyword  25.000000  30.0000  30.0000
137   Default   7.000000      publication   7.000000  29.0000  29.0000
0     Default  11.000000         abstract  11.000000  28.0000  28.0000
172   Default   5.000000            study   5.000000  27.0000  27.0000
105   Default   7.000000            model   7.000000  26.0000  26.0000
60    Default   5.000000       extraction   5.000000  25.0000  25.0000
195   Default   6.000000              use   6.000000  24.0000  24.0000
102   Default   9.000000           method   9.000000  23.0000  23.0000
1     Default   7.000000         academic   7.000000  22.0000  22.0000
69    Default   5.000000         generate   5.000000  21.0000  21.0000
181   Default   5.000000             text   5.000000  20.0000  20.0000
158   Default   4.000000           search   4.000000  19.0000  19.0000
186   Default   4.000000            topic   4.000000  18.0000  18.0000
115   Default   4.000000            paper   4.000000  17.0000  17.0000
12    Default   4.000000             base   4.000000  16.0000  16.0000
44    Default   2.000000       discipline   2.000000  15.0000  15.0000
159   Default   2.000000          section   2.000000  14.0000  14.0000
134   Default   3.000000          propose   3.000000  13.0000  13.0000
152   Default   5.000000           result   5.000000  12.0000  12.0000
100   Default   2.000000            match   2.000000  11.0000  11.0000
150   Default   2.000000         research   2.000000  10.0000  10.0000
201   Default   2.000000             word   2.000000   9.0000   9.0000
185   Default   3.000000            title   3.000000   8.0000   8.0000
151   Default   2.000000       researcher   2.000000   7.0000   7.0000
130   Default   2.000000          problem   2.000000   6.0000   6.0000
104   Default   2.000000          mixture   2.000000   5.0000   5.0000
70    Default   2.000000       generation   2.000000   4.0000   4.0000
139   Default   2.000000          quickly   2.000000   3.0000   3.0000
126   Default   2.000000         previous   2.000000   2.0000   2.0000
169   Default   2.000000            solve   2.000000   1.0000   1.0000
...       ...        ...              ...        ...      ...      ...
51    Topic10   0.002483        empirical   1.141332   0.5591  -5.2901
135   Topic10   0.002358        prototype   1.087170   0.5561  -5.3418
149   Topic10   0.002453   representation   1.134258   0.5532  -5.3022
88    Topic10   0.002477         infinite   1.145902   0.5529  -5.2923
9     Topic10   0.002495      approximate   1.155161   0.5522  -5.2850
14    Topic10   0.002456         bayesian   1.137147   0.5520  -5.3009
43    Topic10   0.002479        dirichlet   1.149303   0.5506  -5.2916
61    Topic10   0.002468        filtering   1.147430   0.5478  -5.2961
87    Topic10   0.002465        inference   1.152985   0.5418  -5.2973
98    Topic10   0.002428              lsi   1.136404   0.5411  -5.3125
107   Topic10   0.002528          network   1.230203   0.5021  -5.2721
177   Topic10   0.002521  systematization   1.226716   0.5023  -5.2748
163   Topic10   0.002507         sentence   1.224030   0.4991  -5.2802
74    T

In [73]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
#print(df_topic_keywords)


In [74]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,prevent,research,online,filtering,work,graph,differently,candidate,structure,problem,increasing,produce,fuzzy,handle,divide
Topic 1,keyword,model,abstract,paper,title,summary,help,extract,result,information,include,prototype,short,save,researcher
Topic 2,keyword,abstract,publication,use,study,academic,method,search,text,discipline,section,result,quickly,measure,generate
Topic 3,keyword,extraction,method,paper,generate,base,research,match,present,approach,graph,propose,unsupervised,new,academic
Topic 4,year,situation,score,lda,ccg,prototype,development,algorithm,variational,different,make,probability,select,context,successful
Topic 5,model,topic,text,mixture,set,underlying,probabilistic,collection,document,probability,lda,level,unigram,lsi,collaborative
Topic 6,researcher,methodology,include,node,title,important,introduction,data,technique,special,representation,implement,baye,use,year
Topic 7,model,abstract,publication,use,study,keyword,academic,method,result,datum,text,topic,lda,training,empirical
Topic 8,derivation,hard,online,classification,result,network,systematization,ofabstract,wepresent,prevent,sentence,graphbase,turkey,modeling,lda
Topic 9,corpora,ccg,unigram,turn,estimation,discipline,produce,perceptron,paramet,usually,complex,relate,focus,future,title


In [77]:
df_document_topic.iloc[2][3]

0.99

In [78]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["my favourite topic is image processing"]
topic, prob_scores = predict_topic(text = mytext)
print(" about the topics")
print(topic)
print(" probability scores")
print(prob_scores)

TypeError: 'list' object is not callable

In [89]:
list=[]
list=df_topic_keywords.iloc[int(df_document_topic.iloc[2][10])].tolist()
list

['keyword',
 'extraction',
 'method',
 'paper',
 'generate',
 'base',
 'research',
 'match',
 'present',
 'approach',
 'graph',
 'propose',
 'unsupervised',
 'new',
 'academic']

In [80]:
!pip install -U nltk


Requirement already up-to-date: nltk in /home/nbuser/anaconda3_420/lib/python3.5/site-packages (3.4.5)
You should consider upgrading via the 'pip install --upgrade pip' command.


In [31]:
!pip install -U rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=10089 sha256=4e0c1f8e4fdddc7258c29916589de1cb9f61f4f8b185a92c914a908e96d19262
  Stored in directory: /home/nbuser/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
You should consider upgrading via the 'pip install --upgrade pip' command.


In [90]:
def convert(list): 
      
    # Converting integer list to string list 
    s = [str(i) for i in list] 
      
    # Join list items using join() 
    res = (" ".join(s)) 
    
    return(res) 
  

convert_list=convert(list)

In [91]:
from rake_nltk import Rake

r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
r.extract_keywords_from_text(convert_list)
    #r.get_ranked_phrases_with_scores() # To get keyword phrases ranked highest to lowest.
print(r.get_word_degrees())
print(r.get_word_frequency_distribution())

defaultdict(<function Rake._build_word_co_occurance_graph.<locals>.<lambda> at 0x7fe5b41298c8>, {'method': 15, 'paper': 15, 'base': 15, 'keyword': 15, 'extraction': 15, 'research': 15, 'approach': 15, 'graph': 15, 'unsupervised': 15, 'match': 15, 'present': 15, 'new': 15, 'generate': 15, 'propose': 15, 'academic': 15})
Counter({'method': 1, 'paper': 1, 'base': 1, 'keyword': 1, 'extraction': 1, 'research': 1, 'approach': 1, 'new': 1, 'unsupervised': 1, 'match': 1, 'present': 1, 'graph': 1, 'generate': 1, 'propose': 1, 'academic': 1})


In [92]:
list

['keyword',
 'extraction',
 'method',
 'paper',
 'generate',
 'base',
 'research',
 'match',
 'present',
 'approach',
 'graph',
 'propose',
 'unsupervised',
 'new',
 'academic']

In [93]:
r.extract_keywords_from_text(df['abstract'][2])
    #r.get_ranked_phrases_with_scores() # To get keyword phrases ranked highest to lowest.

print(r.get_word_degrees())
print(r.get_word_frequency_distribution())

defaultdict(<function Rake._build_word_co_occurance_graph.<locals>.<lambda> at 0x7fe5b47920d0>, {'paper': 7, 'approaches': 3, 'well': 1, 'extraction': 12, 'complex': 2, 'elaborated': 1, 'graph': 5, 'provides': 3, 'supervised': 1, 'unsupervised': 5, 'keyword': 14, 'survey': 1, 'methods': 5, 'gathers': 2, 'graphbased': 2, 'work': 2, 'addition': 1, 'proposed': 1, 'method': 4, 'development': 1, 'existing': 2, 'nodes': 2, 'review': 2, 'based': 6, 'research': 4, 'candidates': 2, 'extracts': 2, 'special': 2, 'related': 2, 'guidelines': 3, 'future': 2, 'croatian': 3, 'presents': 2, 'new': 5, 'emphasis': 2, 'selectivity': 1, 'task': 3, 'network': 2, 'systematization': 1, 'comprehensive': 2})
Counter({'keyword': 5, 'extraction': 4, 'methods': 3, 'paper': 3, 'approaches': 2, 'graph': 2, 'unsupervised': 2, 'based': 2, 'research': 2, 'new': 2, 'method': 1, 'presents': 1, 'complex': 1, 'candidates': 1, 'emphasis': 1, 'provides': 1, 'supervised': 1, 'survey': 1, 'croatian': 1, 'gathers': 1, 'graphbas

In [94]:
def checkKey(dict, key): 
      
    if key in dict.keys(): 
        print(key,end=", ")
        print("Present, ", end =" ") 
        print("value =", dict[key]) 
        return dict[key]
    else: 
        print("Not present") 
        return 0

In [95]:
final_keywords=[]
for i in range(0,len(list)):
    result=checkKey(r.get_word_degrees(),list[i])
    if(result>2):
        final_keywords.append(list[i])
        
    

keyword, Present,  value = 14
extraction, Present,  value = 12
method, Present,  value = 4
paper, Present,  value = 7
Not present
Not present
research, Present,  value = 4
Not present
Not present
Not present
graph, Present,  value = 5
Not present
unsupervised, Present,  value = 5
new, Present,  value = 5
Not present


In [96]:
print(final_keywords)

['keyword', 'extraction', 'method', 'paper', 'research', 'graph', 'unsupervised', 'new']
